### Model Building Using Ridge Regression 

#### Approach: 
Ridge regression with positive & negative constraints is being used to keep coefficients of certain features in the positive domain & negative domain respectively.

Objective here is to keep the impact of promotions & display campaigns in positive domain & impact of price in a negative domain for given ppg.

In [333]:
# Import necessary librabries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error


import warnings
warnings.filterwarnings("ignore")

In [2]:
# Reading retailer*ppg level data
retailer_ppg_groupby = pd.read_csv("./Retailer_ppg_data.csv")

In [3]:
retailer_ppg_groupby.shape

(1337, 16)

In [4]:
retailer_ppg_groupby['Unique_key'] = retailer_ppg_groupby['Retailer']+'_'+ retailer_ppg_groupby['PPG']

In [5]:
req_comp = retailer_ppg_groupby['Unique_key'].unique()
req_comp

array(['Retailer 0_Promo.Group 19', 'Retailer 0_Promo.Group 20',
       'Retailer 0_Promo.Group 3', 'Retailer 0_Promo.Group 5',
       'Retailer 0_Promo.Group 6', 'Retailer 1_Promo.Group 19',
       'Retailer 1_Promo.Group 20', 'Retailer 1_Promo.Group 3',
       'Retailer 1_Promo.Group 5', 'Retailer 1_Promo.Group 6',
       'Retailer 0_Promo.Group 22', 'Retailer 1_Promo.Group 22'],
      dtype=object)

In [6]:
retailer_ppg_groupby.head()

,Date,Retailer,PPG,Wtd.Selling.Dist,Vol.Sales,promo_catalogue,promo_ins,display_platinum,display_gold,display_silver,display_bronze,Weighted_Price,Week,Year,Unique_key,Month
0,2018-08-05,Retailer 0,Promo.Group 19,98.300000,8.85976,0.0,1.0,0.0,0.0,0.0,0.0,1.000000,<pandas.core.indexes.accessors.DatetimePropert...,2018,Retailer 0_Promo.Group 19,8
1,2018-08-05,Retailer 0,Promo.Group 20,98.433333,3.71096,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,<pandas.core.indexes.accessors.DatetimePropert...,2018,Retailer 0_Promo.Group 20,8
2,2018-08-05,Retailer 0,Promo.Group 3,66.762500,10.65029,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,<pandas.core.indexes.accessors.DatetimePropert...,2018,Retailer 0_Promo.Group 3,8
3,2018-08-05,Retailer 0,Promo.Group 5,75.675000,23.34048,0.0,1.0,0.0,0.0,0.0,0.0,0.250000,<pandas.core.indexes.accessors.DatetimePropert...,2018,Retailer 0_Promo.Group 5,8
4,2018-08-05,Retailer 0,Promo.Group 6,89.618182,55.69276,1.0,0.0,0.0,0.0,1.0,0.0,0.090909,<pandas.core.indexes.accessors.DatetimePropert...,2018,Retailer 0_Promo.Group 6,8


In [7]:
retailer_ppg_groupby['Date'] = pd.to_datetime(retailer_ppg_groupby['Date'])
retailer_ppg_groupby['Week'] = retailer_ppg_groupby['Date'].dt.week

C:\Users\mangesh.shinde\AppData\Local\Temp\ipykernel_23584\1835666306.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  retailer_ppg_groupby['Week'] = retailer_ppg_groupby['Date'].dt.week


In [17]:
retailer_ppg_groupby.columns

Index(['Date', 'Retailer', 'PPG', 'Wtd.Selling.Dist', 'Vol.Sales',
       'promo_catalogue', 'promo_ins', 'display_platinum', 'display_gold',
       'display_silver', 'display_bronze', 'Weighted_Price', 'Week', 'Year',
       'Unique_key', 'Month'],
      dtype='object')

In [19]:

retailer_ppg_groupby['Wtd.Selling.Dist'].max()

99.93333333333334

In [353]:
# Creating helper functions


def get_comp_data(df, ppg, req_comp):
    """
    Creates dataframe with competitor of given PPG
    
    Args: Dataframe with data for all ppgs, PPG, Competitor PPG
    
    """
    
    temp_df = df[df['Unique_key']==ppg]
    print(ppg)
    for i in req_comp:
        if i == ppg:
            continue
        if i == 'NA':
            return temp_df
        comp_df = df[df['Unique_key']==i]
        comp_str = i
        rename_dict = {"Wtd.Selling.Dist":comp_str+"_Wtd.Selling.Dist", "promo_catalogue":comp_str+"_promo_catalogue",
                       "promo_ins":comp_str+"_promo_ins", "display_platinum":comp_str+"_display_platinum",
                       "display_silver":comp_str+"_display_silver","display_bronze":comp_str+"_display_bronze",
                       "Weighted_Price":comp_str+"_Weighted_Price"
                      }
        
        comp_df = comp_df.rename(columns=rename_dict)
        
        temp_df = temp_df.merge(comp_df[['Date',comp_str+"_Wtd.Selling.Dist", comp_str+"_promo_catalogue",comp_str+"_Weighted_Price",
            comp_str+"_promo_ins", comp_str+"_display_platinum", comp_str+"_display_silver",comp_str+"_display_bronze"]], 
                                on =['Date'], how ='left')
        
    return temp_df


def seasonal_index(ppg_df):
    """
    Calculate seasonality at of Vol.Sales at weekly level for each PPG at retailer level
    Args: Retailer*ppg level Dataframe 
    
    Output: Dataframe with seasonal index
    """
    df= ppg_df.groupby(['Year','Week'], as_index=False)['Vol.Sales'].sum()
    df_1 = df.groupby(['Year'], as_index=False)['Vol.Sales'].mean()
    df_1.rename(columns={'Vol.Sales':'Avg_weekly_sales'}, inplace=True)
    df_2 = df.merge(df_1, on ='Year', how='left')
    # df_2 = ppg3_0_df.merge(df_1, on = ['Year','Month'], how = 'left')
    df_2['Seasonal_index'] = df_2['Vol.Sales']/df_2['Avg_weekly_sales']
    df_3 = df_2[['Year','Week','Seasonal_index']]
    df_3 = df_3.groupby('Week',as_index = False)['Seasonal_index'].mean()
    
    df_4 = ppg_df.merge(df_3, on =['Week'], how ='left')
    
    return df_4        

# Create function for VIF calculation

def vif_calculation(df, thres = 10):
    """
    Excludes features with vif higher than the specified threshold.

    Args:
        df (pd.DataFrame): A DataFrame containing features and its values
        threshold (float): The significance level (default is 10).

    Returns:
        pd.DataFrame: A modified DataFrame with excluded features.
    """
    x_vif = df.drop(columns = ['Date', 'Retailer', 'PPG','Vol.Sales','Unique_key', 'Year', 'Month','Week'])
    y = df['Vol.Sales']


    while True:
        Cols = range(x_vif.shape[1])

        vif = np.array([variance_inflation_factor(x_vif.values, i) for i in Cols])
        if all(vif < thres):
            break
        else:
            Cols = np.delete(Cols,np.argmax(vif))
            x_vif = x_vif.iloc[:,Cols]
            
    return x_vif


def run_OLS(x, y):
    """
    Run OLS on given data
    
    Args: Features(x) & target variable(y)
    
    Ouptut: Model Result
    """

    # adding the constant term
    x = x.fillna(0)
    x = sm.add_constant(x)
#     print(x.columns)

    # performing the regression
    # and fitting the model
    result = sm.OLS(y, x).fit()

    return result

def exclude_high_p_values(df, threshold=0.05):
    """
    Excludes features with p-values higher than the specified threshold.

    Args:
        df (pd.DataFrame): A DataFrame containing features and their p-values.
        threshold (float): The significance level (default is 0.05).

    Returns:
        pd.DataFrame: A modified DataFrame with excluded features.
    """
        
    excluded_features = df[df['pvalue'] > threshold]['feature'].tolist()
    filtered_df = df[~df['feature'].isin(excluded_features)]

    return filtered_df



def ols_with_feature_selection(df, ppg_retailer:str, threshold=0.05, thres=10):
    """
    Create model summary of ols with feature selection
    
    Args: 1.PPG Dataframe (df), 2. threshold for p value (threshold), 3. threshold for vif (thres)
    4. ppg_retailer : name of ppg & retailer
    
    Output: Final model summary 
    """
    df = df.fillna(0)
    x_vif = vif_calculation(df,thres=thres)
    x = x_vif
#     print(x_vif.columns)

    y = df['Vol.Sales']
    result = run_OLS(x,y)

    pVals = result.pvalues
    pValue_df = pd.DataFrame(pVals, columns=['pvalue']).reset_index(names = ['feature'])

    pvals_filtered = exclude_high_p_values(pValue_df, threshold=threshold)
    pvals_filtered['feature']

    fixed_features = ['Wtd.Selling.Dist','Weighted_Price','promo_catalogue', 'promo_ins', 'display_platinum','display_gold', 
                      'display_silver', 'display_bronze','Seasonal_index']
    req_features = list(set(pvals_filtered['feature']) | set(fixed_features))

#     req_features = [x for x in req_features if x != 'const'] 
    
#     print(req_features)
    
    x = df[req_features]
    y = df['Vol.Sales']
    
    # Run OLS model
    result = run_OLS(x,y)
    # print model summary
#     print(result.summary())
    
    # saving model summary
    result_df = pd.DataFrame(result.params, columns=['Coeff']).reset_index(names='Features')
    result_df.to_csv(f"../TPO Assignment/model_results/{ppg_retailer}_summary1.csv", index=False)

    return result
    
    
def plot_df(df, x, y, title="", xlabel='Date', ylabel='Val Sales', dpi=100):
    plt.figure(figsize=(15,4), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()
    
    
def ridge_with_constraints(X, y, positive_features, negative_features, alpha, positive_alpha, negative_alpha):
    """
    Ridge Regression with positive and negative constraints on specific features.

    Parameters:
    - X: Feature matrix
    - y: Target vector
    - positive_features: List of indices corresponding to features with positive constraints
    - negative_features: List of indices corresponding to features with negative constraints
    - alpha: Regularization strength for all features (Ridge regularization)
    - positive_alpha: Additional regularization strength for positive-constrained features
    - negative_alpha: Additional regularization strength for negative-constrained features

    Returns:
    - ridge_model: Trained Ridge Regression model
    - features: list of features used for model
    """
    ridge_model = Ridge(alpha=alpha)

    # Create a weight vector for the regularization term
    sample_weight = np.ones(X.shape[0] + X.shape[1])

    # Add additional penalties for positive-constrained features
    sample_weight[X.shape[0]:] = positive_alpha * np.isin(np.arange(X.shape[1]), positive_features)
    
    # Subtract additional penalties for negative-constrained features
    sample_weight[X.shape[0]:] = sample_weight[X.shape[0]:] - negative_alpha * np.isin(np.arange(X.shape[1]), negative_features)
    
    # Fit the Ridge model
    ridge_model.fit(X, y, sample_weight=sample_weight[:X.shape[0]])
    
    # Get feature names
    feature_names = X.columns

    return ridge_model, feature_names


def ridge_regression_with_feature_selection(df, positive_feature_indices:list, negative_feature_indices:list, alpha_regular:float, 
                         alpha_positives:float, alpha_negatives:float, thres:float, ppg_retailer:str, threshold=0.05):
    """
    Run Ridge Regression with feature selection.

    Parameters:
    - df: Dataframe with final data
    - positive_feature_indices: List of indices corresponding to features with positive constraints
    - negative_feature_indices: List of indices corresponding to features with negative constraints
    - alpha_regular: Regularization strength for all features (Ridge regularization)
    - alpha_positives: Additional regularization strength for positive-constrained features
    - alpha_negatives: Additional regularization strength for negative-constrained features
    - thres: Threshold for vif selection
    - threshold: threshold for p-value selection
    - ppg_retailer: Name of the ppg_retialer
    Returns:
    - ridge_model: Trained Ridge Regression model summary
    """

    df = df.fillna(0)
    
    # vif based feature selection
    x_vif = vif_calculation(df,thres=thres)
    x = x_vif
    y = df['Vol.Sales']
    result = run_OLS(x,y)

    # P-value based feature selection
    pVals = result.pvalues
    pValue_df = pd.DataFrame(pVals, columns=['pvalue']).reset_index(names = ['feature'])

    pvals_filtered = exclude_high_p_values(pValue_df, threshold=threshold)
    pvals_filtered['feature']

    fixed_features = ['promo_catalogue', 'promo_ins', 'display_platinum','display_gold', 
                      'display_silver', 'display_bronze','Wtd.Selling.Dist','Weighted_Price','Seasonal_index']
    req_features = list(set(fixed_features) | set(pvals_filtered['feature']))
    sorted_req_features = sorted(req_features, key=lambda x: (fixed_features.index(x) if x in fixed_features else len(fixed_features), x))
    sorted_req_features = [value for value in sorted_req_features if value not in ['const']]

    x = df[sorted_req_features]

    
    # Splitting data into train & test dataset
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) 
    

    # Define indices of features with positive constraints
    positive_feature_indices = positive_feature_indices

    # Define indices of features with negative constraints
    negative_feature_indices = negative_feature_indices

    # Regularization strength for all features
    alpha_regular = alpha_regular

    # Additional regularization strength for positive-constrained features
    alpha_positive = alpha_positives

    # Additional regularization strength for negative-constrained features
    alpha_negative = alpha_negatives

    # Train Ridge Regression model with positive and negative constraints
    ridge_model, feature_names = ridge_with_constraints(X_train, y_train, positive_feature_indices, negative_feature_indices, alpha_regular, alpha_positive, alpha_negative)
    
    # Access the coefficients
    beta_coefficients = ridge_model.coef_
    # Access the intercept
    intercept_value = ridge_model.intercept_
    intercept_row = {'Feature': 'Intercept', 'Coefficient': intercept_value}
    intercept_df = pd.DataFrame([intercept_row])

    # Map coefficients to feature names
    coefficients_with_names = list(zip(feature_names, beta_coefficients))

    result_df = pd.DataFrame(coefficients_with_names, columns=['Feature', 'Coefficient'])


    # Use the trained model for prediction or other tasks
    y_pred = ridge_model.predict(X_test)
    
    # Calculate mean absolute error   
    mae = mean_absolute_error(y_test, y_pred)
    print("Mean Absolute Error (MAE):", mae)
    
    smape = calculate_smape_sklearn(y_test, y_pred)
    print("Symmetric Mean Absolute Percentage Error (SMAPE):", smape)
    
    # Calculate r2_score
    r2 = r2_score(y_test, y_pred)
    print("R-squared (R2) Score:", r2)
    
    
    # Append intercept to result summary
    result_df = pd.concat([result_df, intercept_df], axis=0, ignore_index=True)
    result_df.to_csv(f"../TPO Assignment/model_results/{ppg_retailer}_result.csv", index=False)
    
    return result_df


def calculate_smape_sklearn(actual, predicted):
    """
    Calculate Symmetric Mean Absolute Percentage Error (SMAPE) using sklearn.

    Parameters:
    - actual: List or array of actual values
    - predicted: List or array of predicted values

    Returns:
    - smape: Symmetric Mean Absolute Percentage Error
    """
    actual = [float(val) for val in actual]
    predicted = [float(val) for val in predicted]

    numerator = 2 * np.abs(np.array(predicted) - np.array(actual))
    denominator = np.abs(np.array(predicted)) + np.abs(np.array(actual))

    # Handle the case where both actual and predicted values are zero
    mask = denominator == 0
    denominator[mask] = 1  # Set to 1 to avoid division by zero
    smape = np.mean(numerator / denominator) * 100

    return smape

### Ridge Regression based Model


In [354]:
# Defining indices for positive & negative features
positive_feature_indices = [0,1,2,3,4,5,6]
negative_feature_indices = [7]

##### PPG3_0

In [355]:
ppg3_0_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 0_Promo.Group 3', req_comp=req_comp)
ppg3_0_df_seasonality = seasonal_index(ppg3_0_df)

result_df_3_0 = ridge_regression_with_feature_selection(ppg3_0_df_seasonality, positive_feature_indices, negative_feature_indices, alpha_regular=10, 
                                 alpha_positives= 10000, alpha_negatives=10, thres=10, ppg_retailer='ppg3_0')

result_df_3_0

Mean Absolute Error (MAE): 3.3460053286548166
Symmetric Mean Absolute Percentage Error (SMAPE): 22.88625893623957
R-squared (R2) Score: 0.6720158583832468


,Feature,Coefficient
0,promo_catalogue,5.037532
1,promo_ins,0.827052
2,display_platinum,4.553257
3,display_gold,3.407508
4,display_silver,-0.194145
5,display_bronze,-0.324336
6,Wtd.Selling.Dist,0.239816
7,Weighted_Price,-0.150075
8,Seasonal_index,13.641159
9,Retailer 0_Promo.Group 20_display_platinum,3.646203


##### PPG3_1

In [356]:
ppg3_1_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 1_Promo.Group 3', req_comp=req_comp)
ppg3_1_df_seasonality = seasonal_index(ppg3_1_df)

result_df_3_1 = ridge_regression_with_feature_selection(ppg3_1_df_seasonality, positive_feature_indices, negative_feature_indices, alpha_regular=10, 
                                 alpha_positives= 100000, alpha_negatives=10, thres=10, ppg_retailer='ppg3_1')

result_df_3_1

Mean Absolute Error (MAE): 7.899705977396036
Symmetric Mean Absolute Percentage Error (SMAPE): 28.480165417605
R-squared (R2) Score: 0.6917072261570638


,Feature,Coefficient
0,promo_catalogue,10.942067
1,promo_ins,1.571717
2,display_platinum,9.418158
3,display_gold,1.516262
4,display_silver,2.350832
5,display_bronze,1.056432
6,Wtd.Selling.Dist,0.062624
7,Weighted_Price,-0.533774
8,Seasonal_index,10.023725
9,Retailer 1_Promo.Group 19_display_silver,1.207465


##### PPG5_1

In [357]:
ppg5_1_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 1_Promo.Group 5', req_comp=req_comp)
ppg5_1_df_seasonality = seasonal_index(ppg5_1_df)
result_df_5_1 = ridge_regression_with_feature_selection(ppg5_1_df_seasonality, positive_feature_indices, negative_feature_indices, alpha_regular=10, 
                                 alpha_positives= 100, alpha_negatives=10, thres=10, ppg_retailer='ppg5_1')

result_df_5_1

Mean Absolute Error (MAE): 4.9740956886016106
Symmetric Mean Absolute Percentage Error (SMAPE): 18.284185507293053
R-squared (R2) Score: 0.33563415611310843


,Feature,Coefficient
0,promo_catalogue,3.862131
1,promo_ins,0.000000
2,display_platinum,1.471159
3,display_gold,1.237042
4,display_silver,0.419856
5,display_bronze,1.304083
6,Wtd.Selling.Dist,0.011504
7,Weighted_Price,-0.670528
8,Seasonal_index,5.122859
9,Retailer 0_Promo.Group 20_promo_ins,0.378355


##### PPG5_0

In [358]:
ppg5_0_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 0_Promo.Group 5', req_comp=req_comp)
ppg5_0_df_seasonality = seasonal_index(ppg5_0_df)

result_df_5_0 = ridge_regression_with_feature_selection(ppg5_0_df_seasonality, positive_feature_indices, negative_feature_indices, alpha_regular=10, 
                                 alpha_positives= 100, alpha_negatives=10, thres=10, ppg_retailer='ppg5_0')

result_df_5_0

Mean Absolute Error (MAE): 2.7614272300185716
Symmetric Mean Absolute Percentage Error (SMAPE): 17.270730472169326
R-squared (R2) Score: 0.504737894884888


,Feature,Coefficient
0,promo_catalogue,0.870723
1,promo_ins,3.163952
2,display_platinum,0.000000
3,display_gold,0.000000
4,display_silver,0.000000
5,display_bronze,0.315469
6,Wtd.Selling.Dist,0.054019
7,Weighted_Price,-0.479593
8,Seasonal_index,4.274572
9,Retailer 0_Promo.Group 19_display_bronze,0.323008


##### PPG6_0

In [359]:
ppg6_0_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 0_Promo.Group 6', req_comp=req_comp)
ppg6_0_df_seasonality = seasonal_index(ppg6_0_df)

result_df_6_0 = ridge_regression_with_feature_selection(ppg6_0_df_seasonality, positive_feature_indices, negative_feature_indices, alpha_regular=10, 
                                 alpha_positives= 100, alpha_negatives=10, thres=10, ppg_retailer='ppg6_0')

result_df_6_0

Mean Absolute Error (MAE): 4.522791316548428
Symmetric Mean Absolute Percentage Error (SMAPE): 16.47072538954686
R-squared (R2) Score: 0.8846878557146279


,Feature,Coefficient
0,promo_catalogue,12.637295
1,promo_ins,1.475594
2,display_platinum,12.540608
3,display_gold,4.111279
4,display_silver,-1.668342
5,display_bronze,-1.688521
6,Wtd.Selling.Dist,-0.075706
7,Weighted_Price,-0.067134
8,Seasonal_index,27.148610
9,Retailer 0_Promo.Group 22_promo_ins,1.138482


##### PPG6_1

In [360]:
ppg6_1_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 1_Promo.Group 6', req_comp=req_comp)
ppg6_1_df_seasonality = seasonal_index(ppg6_1_df)

result_df_6_1 = ridge_regression_with_feature_selection(ppg6_1_df_seasonality, positive_feature_indices, negative_feature_indices, alpha_regular=10, 
                                 alpha_positives= 100, alpha_negatives=10, thres=10, ppg_retailer='ppg6_1')

result_df_6_1

Mean Absolute Error (MAE): 15.171104312363552
Symmetric Mean Absolute Percentage Error (SMAPE): 30.57755271131906
R-squared (R2) Score: 0.7234604505343466


,Feature,Coefficient
0,promo_catalogue,19.119438
1,promo_ins,7.261538
2,display_platinum,10.716713
3,display_gold,10.800930
4,display_silver,6.123238
5,display_bronze,5.108837
6,Wtd.Selling.Dist,0.402948
7,Weighted_Price,-0.330137
8,Seasonal_index,21.053138
9,Retailer 1_Promo.Group 20_display_platinum,10.716713


##### PPG19_0

In [361]:
ppg19_0_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 0_Promo.Group 19', req_comp=req_comp)
ppg19_0_df_seasonality = seasonal_index(ppg19_0_df)

result_df_19_0 = ridge_regression_with_feature_selection(ppg19_0_df_seasonality, positive_feature_indices, negative_feature_indices, alpha_regular=10, 
                                 alpha_positives= 100, alpha_negatives=10, thres=10, ppg_retailer='ppg19_0')

result_df_19_0

Mean Absolute Error (MAE): 0.7896975641832725
Symmetric Mean Absolute Percentage Error (SMAPE): 14.181300008149625
R-squared (R2) Score: 0.8238910102130538


,Feature,Coefficient
0,promo_catalogue,0.191700
1,promo_ins,0.702074
2,display_platinum,0.000000
3,display_gold,0.000000
4,display_silver,0.000000
5,display_bronze,0.191700
6,Wtd.Selling.Dist,-0.115384
7,Weighted_Price,0.000000
8,Seasonal_index,3.095378
9,Retailer 0_Promo.Group 22_Wtd.Selling.Dist,-0.010454


##### PPG19_1

In [362]:
ppg19_1_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 1_Promo.Group 19', req_comp=req_comp)
ppg19_1_df_seasonality = seasonal_index(ppg19_1_df)

result_df_19_1 = ridge_regression_with_feature_selection(ppg19_1_df_seasonality, positive_feature_indices, negative_feature_indices, alpha_regular=10, 
                                 alpha_positives= 100, alpha_negatives=10, thres=10, ppg_retailer='ppg19_1')

result_df_19_1

Mean Absolute Error (MAE): 1.3401099120474027
Symmetric Mean Absolute Percentage Error (SMAPE): 17.783030047816787
R-squared (R2) Score: 0.6781862064224824


,Feature,Coefficient
0,promo_catalogue,1.868187
1,promo_ins,0.000000
2,display_platinum,1.133029
3,display_gold,0.186958
4,display_silver,0.263598
5,display_bronze,0.216230
6,Wtd.Selling.Dist,-0.038324
7,Weighted_Price,0.004724
8,Seasonal_index,4.581783
9,Retailer 0_Promo.Group 20_display_silver,0.559440


##### PPG20_0

In [363]:
ppg20_0_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 0_Promo.Group 20', req_comp=req_comp)
ppg20_0_df_seasonality = seasonal_index(ppg20_0_df)

result_df_20_0 = ridge_regression_with_feature_selection(ppg20_0_df_seasonality, positive_feature_indices, negative_feature_indices, alpha_regular=10, 
                                 alpha_positives= 100, alpha_negatives=10, thres=10, ppg_retailer='ppg20_0')

result_df_20_0

Mean Absolute Error (MAE): 1.8808041410369456
Symmetric Mean Absolute Percentage Error (SMAPE): 28.571803733026307
R-squared (R2) Score: 0.5659811084306885


,Feature,Coefficient
0,promo_catalogue,1.981856
1,promo_ins,0.257774
2,display_platinum,2.605811
3,display_gold,0.493498
4,display_silver,0.160649
5,display_bronze,-0.048095
6,Wtd.Selling.Dist,0.012821
7,Weighted_Price,-0.152609
8,Seasonal_index,5.333611
9,Intercept,1.916607


##### PPG20_1

In [364]:
ppg20_1_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 1_Promo.Group 20', req_comp=req_comp)
ppg20_1_df_seasonality = seasonal_index(ppg20_1_df)

result_df_20_1 = ridge_regression_with_feature_selection(ppg20_1_df_seasonality, positive_feature_indices, negative_feature_indices, alpha_regular=10, 
                                 alpha_positives= 100, alpha_negatives=10, thres=10, ppg_retailer='ppg20_1')

result_df_20_1

Mean Absolute Error (MAE): 2.659277894571354
Symmetric Mean Absolute Percentage Error (SMAPE): 23.40550546915944
R-squared (R2) Score: 0.5675406497267154


,Feature,Coefficient
0,promo_catalogue,3.666452
1,promo_ins,1.908197
2,display_platinum,3.660057
3,display_gold,1.246024
4,display_silver,-0.303491
5,display_bronze,0.006077
6,Wtd.Selling.Dist,0.022985
7,Weighted_Price,-0.464987
8,Seasonal_index,4.883488
9,Retailer 0_Promo.Group 6_promo_catalogue,0.863693


##### PPG22_0

In [365]:
ppg22_0_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 0_Promo.Group 22', req_comp=req_comp)
ppg22_0_df_seasonality = seasonal_index(ppg22_0_df)

result_df_22_0 = ridge_regression_with_feature_selection(ppg22_0_df_seasonality, positive_feature_indices, 
                                                         negative_feature_indices, alpha_regular=10,alpha_positives= 10,
                                                         alpha_negatives=10, thres=12, threshold=0.05, ppg_retailer='ppg22_0')

result_df_22_0

Mean Absolute Error (MAE): 0.7291295366129559
Symmetric Mean Absolute Percentage Error (SMAPE): 61.13560197440753
R-squared (R2) Score: 0.5673161486547351


,Feature,Coefficient
0,promo_catalogue,0.000000
1,promo_ins,0.023459
2,display_platinum,0.000000
3,display_gold,0.000000
4,display_silver,0.000000
5,display_bronze,0.000000
6,Wtd.Selling.Dist,0.034230
7,Weighted_Price,0.000000
8,Seasonal_index,0.140259
9,Retailer 0_Promo.Group 20_display_bronze,-0.029577


##### PPG22_1

In [366]:
ppg22_1_df = get_comp_data(retailer_ppg_groupby, ppg='Retailer 1_Promo.Group 22', req_comp=req_comp)
ppg22_1_df_seasonality = seasonal_index(ppg22_1_df)

result_df_22_1 = ridge_regression_with_feature_selection(ppg22_1_df_seasonality, positive_feature_indices, negative_feature_indices, alpha_regular=10, 
                                 alpha_positives= 100, alpha_negatives=10, thres=10, ppg_retailer='ppg22_1')

result_df_22_1

Mean Absolute Error (MAE): 0.39220669090396004
Symmetric Mean Absolute Percentage Error (SMAPE): 14.906545481043832
R-squared (R2) Score: 0.8234536422553975


,Feature,Coefficient
0,promo_catalogue,0.495582
1,promo_ins,0.000000
2,display_platinum,0.247098
3,display_gold,0.039216
4,display_silver,-0.005549
5,display_bronze,0.223627
6,Wtd.Selling.Dist,0.028616
7,Weighted_Price,0.000000
8,Seasonal_index,0.520199
9,Retailer 1_Promo.Group 20_promo_ins,-0.386309


#### Model Summary

In [401]:
# Creating model summary of all models in one file.
import glob
path = r"./model_results/*.csv"

model_summary = pd.DataFrame()
for fname in glob.glob(path):
    
    df = pd.read_csv(fname)
    
    df_transpose = df.transpose()
    
    headers = df_transpose.iloc[0].values
    df_transpose.columns = headers
    
    df_transpose.drop(index='Feature', axis=0, inplace=True)
    
    ppg_name = "_".join(fname.split('\\')[1].split("_")[:-1])
    df_transpose['PPG'] = ppg_name
    
    model_summary = pd.concat([model_summary,df_transpose],axis=0)
    
model_summary = model_summary.reset_index(drop=True)

In [403]:
# Rearranging columns of model summary file

model_summary = model_summary[['PPG', 'Intercept','Seasonal_index','Wtd.Selling.Dist','Weighted_Price',
                               'display_platinum','display_gold','display_silver','display_bronze',
                               'promo_catalogue','promo_ins','Retailer 0_Promo.Group 22_Wtd.Selling.Dist',
                               'Retailer 0_Promo.Group 22_promo_ins',
                               'Retailer 1_Promo.Group 3_promo_ins',
                               'Retailer 0_Promo.Group 20_display_silver',
                               'Retailer 1_Promo.Group 3_display_bronze',
                               'Retailer 1_Promo.Group 6_promo_catalogue',
                               'Retailer 0_Promo.Group 6_promo_catalogue',
                               'Retailer 1_Promo.Group 3_display_silver',
                               'Retailer 0_Promo.Group 20_display_bronze',
                               'Retailer 0_Promo.Group 20_promo_ins',
                               'Retailer 0_Promo.Group 3_Weighted_Price',
                               'Retailer 0_Promo.Group 3_display_bronze',
                               'Retailer 0_Promo.Group 5_promo_catalogue',
                               'Retailer 0_Promo.Group 6_Weighted_Price',
                               'Retailer 0_Promo.Group 6_Wtd.Selling.Dist',
                               'Retailer 1_Promo.Group 19_Weighted_Price',
                               'Retailer 1_Promo.Group 19_display_silver',
                               'Retailer 1_Promo.Group 20_Weighted_Price',
                               'Retailer 1_Promo.Group 20_Wtd.Selling.Dist',
                               'Retailer 1_Promo.Group 6_Weighted_Price',
                               'Retailer 1_Promo.Group 6_Wtd.Selling.Dist',
                               'Retailer 1_Promo.Group 20_promo_ins',
                               'Retailer 1_Promo.Group 5_Wtd.Selling.Dist',
                               'Retailer 1_Promo.Group 5_promo_catalogue',
                               'Retailer 1_Promo.Group 6_display_silver',
                               'Retailer 0_Promo.Group 20_display_platinum',
                               'Retailer 1_Promo.Group 22_display_bronze',
                               'Retailer 1_Promo.Group 20_display_platinum',
                               'Retailer 0_Promo.Group 19_display_bronze',
                               'Retailer 0_Promo.Group 22_display_bronze',
                               'Retailer 0_Promo.Group 3_promo_ins',
                               'Retailer 1_Promo.Group 20_promo_catalogue'
                              ]]

# Saving Model Summary file
model_summary.to_csv('./model_summary_ridge.csv', index=False)

#### Calculating Contribution & percentage contribution

In [382]:
# Creating list of ppg_retailer
ppg_list = list(model_summary['PPG'])

# Making list of ppg dataframes
df_name_list = [ppg19_0_df_seasonality,
 ppg19_1_df_seasonality,
 ppg20_0_df_seasonality,
 ppg20_1_df_seasonality,
 ppg22_0_df_seasonality,
 ppg22_1_df_seasonality,
 ppg3_0_df_seasonality,
 ppg3_1_df_seasonality,
 ppg5_0_df_seasonality,
 ppg5_1_df_seasonality,
 ppg6_0_df_seasonality,
 ppg6_1_df_seasonality]

In [385]:
# Calcuating contribution of each feature in sales volume

for ppg, j in zip(ppg_list,df_name_list):
    ppg_df = j.copy()
    df_2 = pd.DataFrame()
    df_1 = model_summary[model_summary['PPG']==ppg].reset_index(drop=True)
    df_1 = df_1.dropna(axis=1)
    for i in df_1.columns:
        if i=='PPG' or i == 'Intercept':
            continue
        else:
            df_2[i] = df_1[i][0]*ppg_df[i]   

        
        df_2['PPG'] = ppg
        df_2['Date'] = ppg_df['Date']
        df_2['Year'] = ppg_df['Year']
        df_2['Month'] = ppg_df['Month']
        df_2['Week'] = ppg_df['Week']
        df_2['Intercept'] = df_1['Intercept'][0]
        df_2['Vol.Sales'] = ppg_df['Vol.Sales']
        req_cols = [x for x in df_2.columns if x not in ['PPG','Date','Year','Month', 'Week', 'Vol.Sales']]
    
    df_2['Vol.Sales_pred'] = np.sum(df_2[req_cols], axis = 1)
    # Saving files 
    df_2.to_csv(f"./contribution/{ppg}_contribution_ridge.csv", index=False)

In [392]:
# Calculating Monthly contribution

path_1 = r"./contribution//*.csv"
for fname in glob.glob(path_1):

    ppg_name = "_".join(fname.split('\\')[1].split("_")[:-2])
    df = pd.read_csv(fname)
    
    req_cols_1 = [x for x in df.columns if x not in ['PPG','Date','Year','Month', 'Week']]
    yearly_contri_df = df.groupby(['PPG','Year','Month'], as_index=False)[req_cols_1].sum()
    
    yearly_contri_df.to_csv(f"./contribution/monthly_contri/{ppg_name}_ridge.csv", index=False)

In [399]:
# Calculating monthly percentage distribution

path_2 = r"./contribution//monthly_contri//*.csv"
for fname in glob.glob(path_2):
    ppg = "_".join(fname.split("\\")[-1].split("_")[:-1])
    df = pd.read_csv(fname)
    req_cols_2 = [x for x in df.columns if x not in ['PPG','Date','Year','Month','Vol.Sales','Vol.Sales_pred']]
    for i in req_cols_2:
        df[f'perc {i}'] = df[i]/df['Vol.Sales']*100
    df = df.drop(columns=req_cols_2)
    df.to_csv(f"./contribution/monthly_percnt_contri/{ppg}_contribution_ridge.csv")

In [404]:
model_summary

,PPG,Intercept,Seasonal_index,Wtd.Selling.Dist,Weighted_Price,display_platinum,display_gold,display_silver,display_bronze,promo_catalogue,...,Retailer 1_Promo.Group 5_Wtd.Selling.Dist,Retailer 1_Promo.Group 5_promo_catalogue,Retailer 1_Promo.Group 6_display_silver,Retailer 0_Promo.Group 20_display_platinum,Retailer 1_Promo.Group 22_display_bronze,Retailer 1_Promo.Group 20_display_platinum,Retailer 0_Promo.Group 19_display_bronze,Retailer 0_Promo.Group 22_display_bronze,Retailer 0_Promo.Group 3_promo_ins,Retailer 1_Promo.Group 20_promo_catalogue
0,ppg19_0,13.721385,3.095378,-0.115384,0.0,0.0,0.0,0.0,0.1917,0.1917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ppg19_1,9.79248,4.581783,-0.038324,0.004724,1.133029,0.186958,0.263598,0.21623,1.868187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ppg20_0,1.916607,5.333611,0.012821,-0.152609,2.605811,0.493498,0.160649,-0.048095,1.981856,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ppg20_1,1.0435,4.883488,0.022985,-0.464987,3.660057,1.246024,-0.303491,0.006077,3.666452,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ppg22_0,4.025472,0.140259,0.03423,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ppg22_1,-1.02558,0.520199,0.028616,0.0,0.247098,0.039216,-0.005549,0.223627,0.495582,...,0.016211,0.602585,-0.018487,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ppg3_0,-11.828606,13.641159,0.239816,-0.150075,4.553257,3.407508,-0.194145,-0.324336,5.037532,...,NaN,NaN,NaN,3.646203,0.032678,NaN,NaN,NaN,NaN,NaN
7,ppg3_1,-1.666682,10.023725,0.062624,-0.533774,9.418158,1.516262,2.350832,1.056432,10.942067,...,NaN,NaN,NaN,NaN,NaN,1.381758,NaN,NaN,NaN,NaN
8,ppg5_0,8.688478,4.274572,0.054019,-0.479593,0.0,0.0,0.0,0.315469,0.870723,...,NaN,NaN,NaN,NaN,NaN,NaN,0.323008,0.547715,NaN,NaN
9,ppg5_1,16.581168,5.122859,0.011504,-0.670528,1.471159,1.237042,0.419856,1.304083,3.862131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.789413,NaN


##### Observations:
    1. All models perform well on the training data but exhibit average performance on the test data, likely due to the limited size of the dataset.
    2. Model evaluation employs R2 score, Mean Absolute Error (MAE), and Symmetrical Mean Absolute Percentage Error (SMAPE) as metrics to gauge performance.
    3. The objective to ensure coefficients of specific features in the positive domain and price in the negative domain isn't consistently achieved. Certain models showcase unexpected coefficients (e.g., display silver & display bronze with negative coefficients), while the model for PPG19_1 displays a positive coefficient for price, contradicting established business logic.
    4. Bayesian regression offers a potential solution to these challenges by allowing incorporation of prior information and uncertainty estimation in coefficients, enhancing interpretability and addressing issues related to small datasets and inconsistent coefficient signs.